<a href="https://colab.research.google.com/github/Gilgamesh60/Optimized-MNIST/blob/main/MNIST_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this notebook I have built a model for implementing a multi digit classification on the given MNIST database.

For this I decided to use a deep CNN model . I have explained the detailed architecture of this CNN later . In order to get the maximum accuracy possible I have also implemented hyperparameter tuning on this CNN using Optuna package.

# Impoting Libraries

In [ ]:
!pip install optuna
import optuna

In [ ]:
from optuna.samplers import TPESampler
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch
import matplotlib.pyplot as plt
import torch.optim as optim
import os
from torch.utils.data import DataLoader,random_split
from torchvision.utils import save_image
from PIL import Image
from torchvision import datasets, transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop
from torch.optim import Adam

# Importing the MNIST dataset

This is the pre-built code in the assignment.This code will create processed and raw image folders. In the processed folder,we have train and test .pt files.These files contain MNIST database in the form of images and labels(for both train and test datasets). We can use torch.load to get this in the form of image tensors and labels from the beforementioned .pt files

In [ ]:
import os
import warnings
import cv2
import torch
from PIL import Image
from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import download_and_extract_archive


class MNISTM(VisionDataset):
    """MNIST-M Dataset.
    """

    resources = [
        ('https://github.com/liyxi/mnist-m/releases/download/data/mnist_m_train.pt.tar.gz',
         '191ed53db9933bd85cc9700558847391'),
        ('https://github.com/liyxi/mnist-m/releases/download/data/mnist_m_test.pt.tar.gz',
         'e11cb4d7fff76d7ec588b1134907db59')
    ]

    training_file = "mnist_m_train.pt"
    test_file = "mnist_m_test.pt"
    classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four',
               '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']

    @property
    def train_labels(self):
        warnings.warn("train_labels has been renamed targets")
        return self.targets

    @property
    def test_labels(self):
        warnings.warn("test_labels has been renamed targets")
        return self.targets

    @property
    def train_data(self):
        warnings.warn("train_data has been renamed data")
        return self.data

    @property
    def test_data(self):
        warnings.warn("test_data has been renamed data")
        return self.data

    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        """Init MNIST-M dataset."""
        super(MNISTM, self).__init__(root, transform, target_transform=target_transform)

        self.train = train

        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError("Dataset not found." +
                               " You can use download=True to download it")

        if self.train:
            data_file = self.training_file
        else:
            data_file = self.test_file

        print(os.path.join(self.processed_folder, data_file))

        self.data, self.targets = torch.load(os.path.join(self.processed_folder, data_file))

    def __getitem__(self, index):
        """Get images and target for data loader.
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], int(self.targets[index])

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.squeeze().numpy(), 'L')
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        """Return size of dataset."""
        return len(self.data)

    @property
    def raw_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'raw')

    @property
    def processed_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'processed')

    @property
    def class_to_idx(self):
        return {_class: i for i, _class in enumerate(self.classes)}

    def _check_exists(self):
        return (os.path.exists(os.path.join(self.processed_folder, self.training_file)) and
                os.path.exists(os.path.join(self.processed_folder, self.test_file)))

    def download(self):
        """Download the MNIST-M data."""

        if self._check_exists():
            return

        os.makedirs(self.raw_folder, exist_ok=True)
        os.makedirs(self.processed_folder, exist_ok=True)

        # download files
        for url, md5 in self.resources:
            filename = url.rpartition('/')[2]
            download_and_extract_archive(url, download_root=self.raw_folder,
                                         extract_root=self.processed_folder,
                                         filename=filename, md5=md5)

        print('Done!')

    def extra_repr(self):
        return "Split: {}".format("Train" if self.train is True else "Test")

# Downloading MNIST dataset.

In [ ]:
p1=MNISTM('MNIST/raw/train-images-idx3-ubyte',download=True)

  0%|          | 0/112143175 [00:00<?, ?it/s]

Extracting MNIST/raw/train-images-idx3-ubyte/MNISTM/raw/mnist_m_train.pt.tar.gz to MNIST/raw/train-images-idx3-ubyte/MNISTM/processed


  0%|          | 0/18663594 [00:00<?, ?it/s]

Extracting MNIST/raw/train-images-idx3-ubyte/MNISTM/raw/mnist_m_test.pt.tar.gz to MNIST/raw/train-images-idx3-ubyte/MNISTM/processed
Done!
MNIST/raw/train-images-idx3-ubyte/MNISTM/processed/mnist_m_train.pt


In [ ]:
#Downloading the processed train and test .pt files
p1.download()

In [ ]:
#Loading the .pt files and store them in train_data and test_data
train_data=torch.load('/content/MNIST/raw/train-images-idx3-ubyte/MNISTM/processed/mnist_m_train.pt')
test_data=torch.load('/content/MNIST/raw/train-images-idx3-ubyte/MNISTM/processed/mnist_m_test.pt')

In [ ]:
train_data 

(tensor([[[[ 60,  77, 133],
           [ 61,  78, 134],
           [ 62,  79, 135],
           ...,
           [ 57,  76, 132],
           [ 57,  76, 132],
           [ 57,  77, 130]],
 
          [[ 60,  77, 133],
           [ 61,  78, 134],
           [ 61,  78, 134],
           ...,
           [ 57,  76, 132],
           [ 57,  76, 132],
           [ 57,  76, 132]],
 
          [[ 61,  78, 134],
           [ 61,  78, 134],
           [ 61,  78, 134],
           ...,
           [ 56,  75, 131],
           [ 56,  75, 131],
           [ 57,  76, 132]],
 
          ...,
 
          [[ 57,  76, 134],
           [ 57,  77, 130],
           [ 57,  77, 130],
           ...,
           [  6,  34,  35],
           [  6,  35,  33],
           [  7,  36,  32]],
 
          [[ 58,  77, 135],
           [ 58,  77, 133],
           [ 57,  77, 130],
           ...,
           [  9,  37,  38],
           [  8,  37,  35],
           [  8,  37,  35]],
 
          [[ 57,  73, 133],
           [ 62,  78

As you can see we have two seperate tensors. One with the collection of all the image arrays and other with the collection of all the corresponding labels. So we need to convert this into suitable format.

In [ ]:
#first element of train_data is the collection of all image arrays
train_data[0]

tensor([[[[ 60,  77, 133],
          [ 61,  78, 134],
          [ 62,  79, 135],
          ...,
          [ 57,  76, 132],
          [ 57,  76, 132],
          [ 57,  77, 130]],

         [[ 60,  77, 133],
          [ 61,  78, 134],
          [ 61,  78, 134],
          ...,
          [ 57,  76, 132],
          [ 57,  76, 132],
          [ 57,  76, 132]],

         [[ 61,  78, 134],
          [ 61,  78, 134],
          [ 61,  78, 134],
          ...,
          [ 56,  75, 131],
          [ 56,  75, 131],
          [ 57,  76, 132]],

         ...,

         [[ 57,  76, 134],
          [ 57,  77, 130],
          [ 57,  77, 130],
          ...,
          [  6,  34,  35],
          [  6,  35,  33],
          [  7,  36,  32]],

         [[ 58,  77, 135],
          [ 58,  77, 133],
          [ 57,  77, 130],
          ...,
          [  9,  37,  38],
          [  8,  37,  35],
          [  8,  37,  35]],

         [[ 57,  73, 133],
          [ 62,  78, 137],
          [ 58,  75, 131],
         

In [ ]:
len(train_data[0]) #60000 training images

60000

In [ ]:
#Second element of train_data is the collection of image digit labels
train_data[1]

tensor([5, 0, 4,  ..., 5, 6, 8])

In [ ]:
len(train_data[1])   #60000 training labels

60000

# Processing the train and test datasets.


Define a function to convert array into tensors and normalize them.

In [ ]:
transform_norm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

As you can see that image shape is (28,28,3).So it is an RGB image. I converted each image to 2D grayscale image of (28,28) shape.

I also generated train and test datasets where each datapoint is in (image tensor , corresponding image label) format.
This is to get the datasets in loader object form for the CNN models.

In [ ]:
train=[]
for i in range(len(train_data[1])):
  arr=np.array(train_data[0][i])
  #grayscaling using opencv
  arr=cv2.cvtColor(arr,cv2.COLOR_BGR2GRAY)
  #appending (image tensor , corresponding image label) tuples.
  train.append((transform_norm(arr),int(train_data[1][i])))
#converting list to tuple
train=tuple(train)

In [ ]:
#Same goes for the test dataset
test=[]
for i in range(len(test_data[1])):
  arr=np.array(test_data[0][i])
  arr=cv2.cvtColor(arr,cv2.COLOR_BGR2GRAY)
  test.append((transform_norm(arr),int(test_data[1][i])))
test=tuple(test)

In [ ]:
print(f"The number of samples in the train set :{len(train)}")
print(f"The number of samples in the test set :{len(test)}")
print(f"The shape of each image tensor in both train and test  :{train[0][0].shape}")

The number of samples in the train set :60000
The number of samples in the test set :10000
The shape of each image tensor in both train and test  :torch.Size([1, 28, 28])


# Building a deep CNN model.


Now for this I have tried many different models. I have only shown the model with the highest accuracy and didn't show any other models.That is because I don't want to make this notebook messy.These are my observations on the accuracy for different models:
1.   LSTM : Approximately 86% accracy over 10 epochs.
2.   RNNs : Approximately 81% accracy over 10 epochs.
3.   Bidirectional LSTM : Approximately 88% accracy over 10 epochs.
4.   Simple CNN : Approximately 83% accracy over 10 epochs.




# ModelM3 CNN model

This is one of the models proposed in this research paper : [https://arxiv.org/abs/2008.10400](https://)

(Note that in this paper they have proposed a much more complex model but due to time limitions I didn't implement this research paper as it is. Instead I have used it to implement some parts of my model.)

Here is a brief introduction to the model proposed in the paper: 
 This network models consist of multiple convolution layers(10) and 2 fully connected layer at the end. In each convolution layer, a 2D convolution is performed, followed by a 2D batch normalization and ReLU activation. Max pooling or average pooling is not used after convolution. Instead, the size of feature map is reduced after each convolution because padding is not used.I have used a 3×3 kernel,so the width and height of the image is reduced by two after each convolution layer.

 In order to avoid the problem of overfitting I have also added a dropout layers before each fully connected layer. Dropout rates are calculated using hyperparameter tuning.

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Hyper-parameters 
# input_size = 784 # 28x28
num_classes = 10
input_size = 784

In [ ]:
class ModelM3(nn.Module):
    def __init__(self,trial):
        super(ModelM3, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, bias=False)       # output becomes 26x26
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 48, 3, bias=False)      # output becomes 24x24
        self.conv2_bn = nn.BatchNorm2d(48)
        self.conv3 = nn.Conv2d(48, 64, 3, bias=False)      # output becomes 22x22
        self.conv3_bn = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 80, 3, bias=False)      # output becomes 20x20
        self.conv4_bn = nn.BatchNorm2d(80)
        self.conv5 = nn.Conv2d(80, 96, 3, bias=False)      # output becomes 18x18
        self.conv5_bn = nn.BatchNorm2d(96)
        self.conv6 = nn.Conv2d(96, 112, 3, bias=False)     # output becomes 16x16
        self.conv6_bn = nn.BatchNorm2d(112)
        self.conv7 = nn.Conv2d(112, 128, 3, bias=False)    # output becomes 14x14
        self.conv7_bn = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(128, 144, 3, bias=False)    # output becomes 12x12
        self.conv8_bn = nn.BatchNorm2d(144)
        self.conv9 = nn.Conv2d(144, 160, 3, bias=False)    # output becomes 10x10
        self.conv9_bn = nn.BatchNorm2d(160)
        self.conv10 = nn.Conv2d(160, 176, 3, bias=False)   # output becomes 8x8
        self.conv10_bn = nn.BatchNorm2d(176)
        dropout_rate = trial.suggest_float("dropout_rate", 0, 0.5,step=0.1)
        self.drop1=nn.Dropout2d(p=dropout_rate)   
        fc2_input_dim =  trial.suggest_int("fc2_input_dim", 32, 128,32)
        self.fc1 = nn.Linear( 11264,fc2_input_dim)
        dropout_rate2 = trial.suggest_float("dropout_rate2", 0, 0.3,step=0.1)
        self.drop2=nn.Dropout2d(p=dropout_rate2)
        self.fc2 = nn.Linear(fc2_input_dim, 10)
        self.fc1_bn = nn.BatchNorm1d(10)
    def forward(self, x):
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        conv5 = F.relu(self.conv5_bn(self.conv5(conv4)))
        conv6 = F.relu(self.conv6_bn(self.conv6(conv5)))
        conv7 = F.relu(self.conv7_bn(self.conv7(conv6)))
        conv8 = F.relu(self.conv8_bn(self.conv8(conv7)))
        conv9 = F.relu(self.conv9_bn(self.conv9(conv8)))
        conv10 = F.relu(self.conv10_bn(self.conv10(conv9)))
        x=  self.drop1(conv10)
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)
        x = self.fc2(x)
        return x

# Hyperparameter tuning the CNN model using Optuna package

Here we will optimize following hyperparameters :
1.   Batch size
2.   Dropout_rate1 for first conneted layer.
3.   Dropout_rate2 for second conneted layer.
4.   fc2_input_dim is the number of output channels for first connected layer


For this we are using a Optuna package. In Optuna, the goal is to minimize/maximize the objective function, which takes as input a set of hyperparameters and returns a validation score. For each hyperparameter, we consider a different range of values.


In [ ]:
# objective function takes trial as an input which contains optimizer,batch_size,learning_rate
def objective(trial):

    # Generate the model.
    model = ModelM3(trial).to(DEVICE)

    # Generate the optimizers.

    # try RMSprop and SGD
    '''
    optimizer_name = trial.suggest_categorical("optimizer", ["RMSprop", "SGD"])
    momentum = trial.suggest_float("momentum", 0.0, 1.0)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr,momentum=momentum)
    '''
    #try Adam, AdaDelta adn Adagrad
    
    #Defining search space for each parameter.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "Adadelta","Adagrad"])  
    lr = trial.suggest_float("lr", 1e-5, 1e-1,log=True)              
    batch_size=trial.suggest_int("batch_size", 64, 256,step=64)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion=nn.CrossEntropyLoss()

    # Converting train and test datasets we obtsained before into dataloader object.
    train_loader=torch.utils.data.DataLoader(train,shuffle=True,batch_size=batch_size,pin_memory=True)
    test_loader=torch.utils.data.DataLoader(test,shuffle=False,batch_size=batch_size,pin_memory=True)
    
    # Training of the model.
    EPOCHS=5
    for epoch in range(EPOCHS):
        model.train()   
       
        for batch_idx, (images, labels) in enumerate(train_loader):
            # Limiting training images for faster epochs.
            #if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
            #    break

            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()    #backpropogation 
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (images, labels) in enumerate(test_loader):
                # Limiting validation images.
               # if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                #    break
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                output = model(images)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(labels.view_as(pred)).sum().item()

        accuracy = correct / len(test_loader.dataset)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

Somethings related to the code below-


*  Sampler : Sampler value indicates which sampler method you want Optuna to implement. I am using TPESampler() which is based on Bayesian hyperparameter optimization, which is an efficient method for hyperparameter tuning.
*   n_trials : Indicates the number of trials in a study that you want Optuna to perform, here it's 20.
*   optimize method : Optimize method in our study will optimize the output of the function objective i.e accuracy of the model.

  At the end we will choose the hyperparameters that give us the best accuracy.








In [ ]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
study.optimize(objective, n_trials=20)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2022-12-04 08:42:09,076] A new study created in memory with name: no-name-d86ac46f-0200-4b50-9e8d-d543f0ac802d
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
[I 2022-12-04 08:49:36,760] Trial 0 finished with value: 0.8721 and parameters: {'dropout_rate': 0.5, 'fc2_input_dim': 128, 'dropout_rate2': 0.0, 'optimizer': 'Adam', 'lr': 0.00011600367730872151, 'batch_size': 192}. Best is trial 0 with value: 0.8721.
[I 2022-12-04 08:56:42,305] Trial 1 finished with value: 0.8094 and parameters: {'dropout_rate': 0.0, 'fc2_input_dim': 64, 'dropout_rate2': 0.1, 'optimizer': 'Ad

Accuracy: 0.9014
Best hyperparameters: {'dropout_rate': 0.30000000000000004, 'fc2_input_dim': 96, 'dropout_rate2': 0.2, 'optimizer': 'Adam', 'lr': 0.0006929852280913366, 'batch_size': 256}


So the best performing hyperparameters to increase our accuracy are:

1.   dropout_rate1 :    0.30000000000000004
2.   fc2_input_dim :    96
3.   dropout_rate2 : 0.2
4.   optimizer:  Adam
5.   batch_size:  256



**I managed to obtain the best hyperparameters that will maximize our model accuracy. Also for tuning I decided to use small number of EPOCHS(only 2) since it was taking too long to run. But now we can run the model on higher number of EPOCHS using the hyperparameters we obtained above.So let's rebuild the model again by substituting abovementioned hyperparameters.**

In [ ]:
class ModelM3(nn.Module):
    def __init__(self):
        super(ModelM3, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, bias=False)       # output becomes 26x26
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 48, 3, bias=False)      # output becomes 24x24
        self.conv2_bn = nn.BatchNorm2d(48)
        self.conv3 = nn.Conv2d(48, 64, 3, bias=False)      # output becomes 22x22
        self.conv3_bn = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 80, 3, bias=False)      # output becomes 20x20
        self.conv4_bn = nn.BatchNorm2d(80)
        self.conv5 = nn.Conv2d(80, 96, 3, bias=False)      # output becomes 18x18
        self.conv5_bn = nn.BatchNorm2d(96)
        self.conv6 = nn.Conv2d(96, 112, 3, bias=False)     # output becomes 16x16
        self.conv6_bn = nn.BatchNorm2d(112)
        self.conv7 = nn.Conv2d(112, 128, 3, bias=False)    # output becomes 14x14
        self.conv7_bn = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(128, 144, 3, bias=False)    # output becomes 12x12
        self.conv8_bn = nn.BatchNorm2d(144)
        self.conv9 = nn.Conv2d(144, 160, 3, bias=False)    # output becomes 10x10
        self.conv9_bn = nn.BatchNorm2d(160)
        self.conv10 = nn.Conv2d(160, 176, 3, bias=False)   # output becomes 8x8
        self.conv10_bn = nn.BatchNorm2d(176)
        dropout_rate =  0.30000000000000004
        self.drop1=nn.Dropout2d(p=dropout_rate)   
        fc2_input_dim =96
        self.fc1 = nn.Linear( 11264,fc2_input_dim)
        dropout_rate2=0.2
        self.drop2=nn.Dropout2d(p=dropout_rate2)
        self.fc2 = nn.Linear(fc2_input_dim, 10)
        self.fc1_bn = nn.BatchNorm1d(10)
    def forward(self, x):
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        conv5 = F.relu(self.conv5_bn(self.conv5(conv4)))
        conv6 = F.relu(self.conv6_bn(self.conv6(conv5)))
        conv7 = F.relu(self.conv7_bn(self.conv7(conv6)))
        conv8 = F.relu(self.conv8_bn(self.conv8(conv7)))
        conv9 = F.relu(self.conv9_bn(self.conv9(conv8)))
        conv10 = F.relu(self.conv10_bn(self.conv10(conv9)))
        x=  self.drop1(conv10)
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)
        x = self.fc2(x)
        return x

# Model training and Evaluation.

In [ ]:
model = ModelM3().to(DEVICE)
learning_rate = 0.0006929852280913366
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
batch_size=256
criterion=nn.CrossEntropyLoss()
#define train_loader and test_loader objects.
train_loader=torch.utils.data.DataLoader(train,shuffle=True,batch_size=batch_size,pin_memory=True)
test_loader=torch.utils.data.DataLoader(test,shuffle=False,batch_size=batch_size,pin_memory=True)
    
# Training of the model.
EPOCHS=15
n_total_steps = len(train_loader) 
for epoch in range(EPOCHS):
      model.train()
       
      for batch_idx, (images, labels) in enumerate(train_loader):

            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()    #backpropogation
            optimizer.step()
            if (batch_idx+1) % 100 == 0:
                  print (f'Epoch [{epoch+1}/{EPOCHS}], Step [{batch_idx+1}/{n_total_steps}],Loss: {loss.item():.4f}')
      #Evaluation on test dataset.
      model.eval()
      correct = 0
      with torch.no_grad():
          for batch_idx, (images, labels) in enumerate(test_loader):
                # Limiting validation images.
               # if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                #    break
              images, labels = images.to(DEVICE), labels.to(DEVICE)
              output = model(images)
               # Get the index of the max log-probability.
              pred = output.argmax(dim=1, keepdim=True)
              correct += pred.eq(labels.view_as(pred)).sum().item()

      accuracy = (correct / len(test_loader.dataset))*100
      print(f'Accuracy of the network on the 10000 test images: {accuracy} %')

Epoch [1/15], Step [100/235],Loss: 0.4802
Epoch [1/15], Step [200/235],Loss: 0.4044
Accuracy of the network on the 10000 test images: 90.11 %
Epoch [2/15], Step [100/235],Loss: 0.3335
Epoch [2/15], Step [200/235],Loss: 0.3554
Accuracy of the network on the 10000 test images: 93.84 %
Epoch [3/15], Step [100/235],Loss: 0.2055
Epoch [3/15], Step [200/235],Loss: 0.2271
Accuracy of the network on the 10000 test images: 95.45 %
Epoch [4/15], Step [100/235],Loss: 0.2351
Epoch [4/15], Step [200/235],Loss: 0.1671
Accuracy of the network on the 10000 test images: 95.54 %
Epoch [5/15], Step [100/235],Loss: 0.0900
Epoch [5/15], Step [200/235],Loss: 0.1374
Accuracy of the network on the 10000 test images: 95.82000000000001 %
Epoch [6/15], Step [100/235],Loss: 0.1167
Epoch [6/15], Step [200/235],Loss: 0.1041
Accuracy of the network on the 10000 test images: 95.98 %
Epoch [7/15], Step [100/235],Loss: 0.0763
Epoch [7/15], Step [200/235],Loss: 0.1539
Accuracy of the network on the 10000 test images: 96

**As you can see this model is getting almost 97% accuracy on the 10000 test images.**

## Saving and loading networks

Most of the times these models take too much time to train. 
So it's impractical to train a network every time you need to use it. Instead, we can save trained networks then load them later to train more or use them for predictions.

The parameters for PyTorch networks are stored in a model's `state_dict`. We can see the state dict contains the weight and bias matrices for each of our layers.Same goes for optimizer.

In [ ]:


print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

Our model: 

 ModelM3(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv1_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv2_bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv3_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 80, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv4_bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(80, 96, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv5_bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(96, 112, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv6_bn): BatchNorm2d(112, eps=1e-05, momentum=0.1, affine=True, 

In [ ]:
print("Our optimizer : \n\n", optimizer, '\n')
print("The state dict keys: \n\n", optimizer.state_dict().keys())

Our optimizer : 

 Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    weight_decay: 0
) 

The state dict keys: 

 dict_keys(['state', 'param_groups'])


In [ ]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, 'checkpoint.pt')
from google.colab import files
# download checkpoint file
files.download('checkpoint.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>